<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Mask_NoMask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install roboflow
import os
import glob
import pandas as pd
from roboflow import Roboflow
import xml.etree.ElementTree as ET

# Data

In [3]:
rf = Roboflow(api_key="")
project = rf.workspace("joseph-nelson").project("mask-wearing")
dataset = project.version(4).download("tensorflow")
os.rename('/content/Mask-Wearing-4', '/content/data')

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Mask-Wearing-4 in tensorflow:: 100%|██████████| 154/154 [00:00<00:00, 464.07it/s]


## XML to CSV
Convert annotation files(xml format) to a csv file.
(for manually annotated imges)


In [4]:
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text))
            
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [5]:
images_path = '/content/'

xml_df = xml_to_csv(images_path)
xml_df.to_csv('/content/annotations.csv', index=None)    

## Label Map

Used in TensorFlow Object Detection API

In [6]:
labelmap_file = '''item {
    id: 1
    name: 'mask'
}
item {
    id: 2
    name: 'no-mask'
}'''

with open('/content/Mask-Wearing-4/labelmap.pbtxt', 'w+') as f:
  f.write(labelmap_file)
f.close()

## Transfer Dataset to TFrecords

### Install Object Detection API


In [ ]:
import pathlib
# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [34]:
import io
import argparse
import collections
from PIL import Image
import tensorflow as tf

from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

def split(df, group):
	data = collections.namedtuple('data', ['filename', 'object'])
	gb = df.groupby(group)
	return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path, category_idx):
	with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
		encoded_jpg = fid.read()
	encoded_jpg_io = io.BytesIO(encoded_jpg)
	image = Image.open(encoded_jpg_io)
	width, height = image.size

	filename = group.filename.encode('utf8')
	image_format = b'jpg'
	xmins = []
	xmaxs = []
	ymins = []
	ymaxs = []
	classes_text = []
	classes = []

	for index, row in group.object.iterrows():
		xmins.append(float(row['xmin']) / width)
		xmaxs.append(float(row['xmax']) / width)
		ymins.append(float(row['ymin']) / height)
		ymaxs.append(float(row['ymax']) / height)
		classes_text.append(row['class'].encode('utf8'))
		classes.append(category_idx[row['class']])

	tf_example = tf.train.Example(features=tf.train.Features(feature={
		'image/height': dataset_util.int64_feature(height),
		'image/width': dataset_util.int64_feature(width),
		'image/filename': dataset_util.bytes_feature(filename),
		'image/source_id': dataset_util.bytes_feature(filename),
		'image/encoded': dataset_util.bytes_feature(encoded_jpg),
		'image/format': dataset_util.bytes_feature(image_format),
		'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
		'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
		'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
		'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
		'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
		'image/object/class/label': dataset_util.int64_list_feature(classes),
	}))
	return tf_example

In [ ]:
for f_name in ['train', 'test', 'valid']:

  img_path = f'/content/data/{f_name}'
  csv_path = f'/content/data/{f_name}/_annotations.csv'
  tfrecord_path = f'/content/data/{f_name}/{f_name}.tfrecord'
  labelmap_path  = '/content/data/labelmap.pbtxt'

  label_map_dict = label_map_util.get_label_map_dict(labelmap_path)
  writer = tf.io.TFRecordWriter(tfrecord_path)

  examples = pd.read_csv(csv_path)
  grouped = split(examples, 'filename')
  for group in grouped:
    tf_example = create_tf_example(group, img_path, label_map_dict)
    writer.write(tf_example.SerializeToString())

  writer.close()
  print(f'Successfully created the TFRecords: {tfrecord_path}')

#Model

## Install a model from the model garden
Choose from a large selection of object detection models in the [TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md).

In [55]:
MODEL = "faster_rcnn_resnet50_v1_640x640_coco17_tpu-8"

%cd /content/models/research/object_detection
!mkdir pre-trained-models
%cd pre-trained-models
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/{MODEL}.tar.gz
!tar -xf {MODEL}.tar.gz
!rm {MODEL}.tar.gz

/content/models/research/object_detection
/content/models/research/object_detection/pre-trained-models
--2022-12-29 17:48:09--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.111.128, 2607:f8b0:4001:c07::80
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.111.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211996178 (202M) [application/x-tar]
Saving to: ‘faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz’

faster_rcnn_resnet5 100%[===================>] 202.17M   128MB/s    in 1.6s    

2022-12-29 17:48:11 (128 MB/s) - ‘faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz’ saved [211996178/211996178]



## Edit pipeline.config
The pipeline.config file is in protocol buffer format, and we can use the config_util to change the settings we need.

In [56]:
from object_detection.utils import config_util


from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [ ]:
pipeline_config = f"./{MODEL}/pipeline.config"
model_dir = f"./{MODEL}/checkpoint"

config = config_util.get_configs_from_pipeline_file(pipeline_config)

model_config = config['model']
model_config.faster_rcnn.num_classes = 2

train_config = config['train_config']

#Starting checkpoin location
train_config.fine_tune_checkpoint = f"/content/models/research/object_detection/pre-trained-models/{MODEL}/checkpoint/ckpt-0"

# GPU memory limits batch size
train_config.batch_size = 32
train_config.use_bfloat16 = False


#...................$$$$$$$$$$$$$$$.................
# some models use "fine_tune" rather than "detection" for fine-tuning
train_config.fine_tune_checkpoint_type = "detection"
# detection      ===> use all the layers as pretrained layers
# classification ===> use only the classification layer as pretrained and initialize the weights randomly


# set auto-augmentation
del train_config.data_augmentation_options[:]

a1=train_config.data_augmentation_options.add()
a1.autoaugment_image.policy_name = "v2"

# training dataset (80%)
train_input_config = config['train_input_config']
train_input_config.label_map_path = "/content/data/labelmap.pbtxt"
train_input_config.tf_record_input_reader.input_path[:] = ["/content/data/train/*.tfrecord",
                                                           "/content/data/eval/*.tfrecord"]  # have little data, so we use eval as train data
# evaluation dataset (20%)                                                           
eval_input_config = config['eval_input_config']
eval_input_config.label_map_path = "/content/data/labelmap.pbtxt"
eval_input_config.tf_record_input_reader.input_path[:] = ["/content/data/test/*.tfrecord"]

# save pipeline.config file
pipeline_proto = config_util.create_pipeline_proto_from_configs(config)
config_util.save_pipeline_config(pipeline_config=pipeline_proto, 
                                 directory=f'/content/models/research/object_detection/pre-trained-models/{MODEL}')